# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [24]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [25]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [26]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [27]:
# How big input layer?
n_features = (X_train.shape[1],)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [28]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(units=100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(units=50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(units=1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 970us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 939us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 970us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 969us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [=======================

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [29]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [30]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [31]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

Epoch 137/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [32]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [33]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.4708 - mse: 0.4708 - val_loss: 0.2227 - val_mse: 0.2227
Epoch 2/150
33/33 [==============================] - 0s 939us/step - loss: 0.2566 - mse: 0.2566 - val_loss: 0.1690 - val_mse: 0.1690
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2206 - mse: 0.2206 - val_loss: 0.1491 - val_mse: 0.1491
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1967 - mse: 0.1967 - val_loss: 0.1369 - val_mse: 0.1369
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1762 - mse: 0.1762 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1612 - mse: 0.1612 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1523 - mse: 0.1523 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1426 -

33/33 [==============================] - 0s 1ms/step - loss: 0.0306 - mse: 0.0306 - val_loss: 0.1589 - val_mse: 0.1589
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0312 - mse: 0.0312 - val_loss: 0.1608 - val_mse: 0.1608
Epoch 65/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0295 - mse: 0.0295 - val_loss: 0.1582 - val_mse: 0.1582
Epoch 66/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0292 - mse: 0.0292 - val_loss: 0.1589 - val_mse: 0.1589
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0286 - mse: 0.0286 - val_loss: 0.1579 - val_mse: 0.1579
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0283 - mse: 0.0283 - val_loss: 0.1610 - val_mse: 0.1610
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0285 - mse: 0.0285 - val_loss: 0.1604 - val_mse: 0.1604
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0279 - 

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.1697 - val_mse: 0.1697
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0148 - mse: 0.0148 - val_loss: 0.1676 - val_mse: 0.1676
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0148 - mse: 0.0148 - val_loss: 0.1667 - val_mse: 0.1667
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0146 - mse: 0.0146 - val_loss: 0.1686 - val_mse: 0.1686
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0146 - mse: 0.0146 - val_loss: 0.1689 - val_mse: 0.1689
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0140 - mse: 0.0140 - val_loss: 0.1688 - val_mse: 0.1688
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0142 - mse: 0.0142 - val_loss: 0.1695 - val_mse: 0.1695
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0121 - mse: 0.0121


[0.012103324756026268, 0.012103324756026268]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [35]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1746 - mse: 0.1746


[0.17463701963424683, 0.17463701963424683]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [36]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

32839.73049902025

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [37]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(units=100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD',
                loss='mse',
                metrics=['mse'])
# Train the model
he_model.fit(X_train,
            y_train_scaled,
            batch_size=32,
            epochs=150,
            validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5256 - mse: 0.5256 - val_loss: 0.2384 - val_mse: 0.2384
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2681 - mse: 0.2681 - val_loss: 0.1788 - val_mse: 0.1788
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2131 - mse: 0.2131 - val_loss: 0.1512 - val_mse: 0.1512
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1817 - mse: 0.1817 - val_loss: 0.1334 - val_mse: 0.1334
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1657 - mse: 0.1657 - val_loss: 0.1232 - val_mse: 0.1232
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1449 - mse: 0.1449 - val_loss: 0.1183 - val_mse: 0.1183
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1365 - mse: 0.1365 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1283 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1461 - val_mse: 0.1461
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1433 - val_mse: 0.1433
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1502 - val_mse: 0.1502
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1470 - val_mse: 0.1470
Epoch 67/150
33/33 [==============================] - 0s 999us/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1470 - val_mse: 0.1470
Epoch 68/150
33/33 [==============================] - 0s 970us/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1469 - val_mse: 0.1469
Epoch 69/150
33/33 [==============================] - 0s 909us/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1463 - val_mse: 0.1463
Epoch 70/150
33/33 [==============================] - 0s 939us/step - loss: 0.0217 

33/33 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1586 - val_mse: 0.1586
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1583 - val_mse: 0.1583
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1574 - val_mse: 0.1574
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1584 - val_mse: 0.1584
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1572 - val_mse: 0.1572
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1584 - val_mse: 0.1584
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0089 -

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [38]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0068 - mse: 0.0068


[0.0067797438241541386, 0.0067797438241541386]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [39]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1595 - mse: 0.1595


[0.15946754813194275, 0.15946754813194275]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [40]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD',
                loss='mse',
                metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4673 - mse: 0.4673 - val_loss: 0.1568 - val_mse: 0.1568
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2506 - mse: 0.2506 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1657 - mse: 0.1657 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1604 - mse: 0.1604 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1434 - mse: 0.1434 - val_loss: 0.1261 - val_mse: 0.1261
Epoch 7/150
33/33 [==============================] - 0s 970us/step - loss: 0.1323 - mse: 0.1323 - val_loss: 0.1107 - val_mse: 0.1107
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1201 -

33/33 [==============================] - 0s 1ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1304 - val_mse: 0.1304
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1312 - val_mse: 0.1312
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 66/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 67/150
33/33 [==============================] - 0s 970us/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.1312 - val_mse: 0.1312
Epoch 68/150
33/33 [==============================] - 0s 940us/step - loss: 0.0195 - mse: 0.0195 - val_loss: 0.1292 - val_mse: 0.1292
Epoch 69/150
33/33 [==============================] - 0s 909us/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 70/150
33/33 [==============================] - 0s 970us/step - loss: 0.01

33/33 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 125/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1381 - val_mse: 0.1381
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1392 - val_mse: 0.1392
Epoch 127/150
33/33 [==============================] - 0s 909us/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 128/150
33/33 [==============================] - 0s 939us/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 129/150
33/33 [==============================] - 0s 909us/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1378 - val_mse: 0.1378
Epoch 130/150
33/33 [==============================] - 0s 970us/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1384 - val_mse: 0.1384
Epoch 131/150
33/33 [==============================] - 0s 939us/step - l

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [41]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070


[0.007024451158940792, 0.007024451158940792]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [42]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1398 - mse: 0.1398


[0.13983385264873505, 0.13983385264873505]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [43]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3191 - mse: 0.3191 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1647 - mse: 0.1647 - val_loss: 0.0896 - val_mse: 0.0896
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1266 - mse: 0.1266 - val_loss: 0.0878 - val_mse: 0.0878
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1109 - mse: 0.1109 - val_loss: 0.0909 - val_mse: 0.0909
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0823 - mse: 0.0823 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 6/150
33/33 [==============================] - 0s 939us/step - loss: 0.0663 - mse: 0.0663 - val_loss: 0.0905 - val_mse: 0.0905
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0680 - mse: 0.0680 - val_loss: 0.0954 - val_mse: 0.0954
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0452 -

33/33 [==============================] - 0s 1ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0817 - val_mse: 0.0817
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.0845 - val_mse: 0.0845
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.0855 - val_mse: 0.0855
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.0903 - val_mse: 0.0903
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.0942 - val_mse: 0.0942
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0796 - val_mse: 0.0796
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.0871 - val_mse: 0.0871
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0068 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0794 - val_mse: 0.0794
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.0767 - val_mse: 0.0767
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0758 - val_mse: 0.0758
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0794 - val_mse: 0.0794
Epoch 129/150
33/33 [==============================] - 0s 910us/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.0802 - val_mse: 0.0802
Epoch 130/150
33/33 [==============================] - 0s 970us/step - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0828 - val_mse: 0.0828
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0796 - val_mse: 0.0796
Epoch 132/150
33/33 [==============================] - 0s 970us/s

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [44]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 636us/step - loss: 0.0068 - mse: 0.0068


[0.006764986552298069, 0.006764986552298069]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [46]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 778us/step - loss: 0.0780 - mse: 0.0780


[0.07795635610818863, 0.07795635610818863]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [47]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3759 - mse: 0.3759 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1458 - mse: 0.1458 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1015 - mse: 0.1015 - val_loss: 0.1142 - val_mse: 0.1142
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0789 - mse: 0.0789 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0607 - mse: 0.0607 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0454 - mse: 0.0454 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0379 - mse: 0.0379 - val_loss: 0.1280 - val_mse: 0.1280
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0297 - m

33/33 [==============================] - 0s 1ms/step - loss: 9.5339e-04 - mse: 9.5339e-04 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 8.9782e-04 - mse: 8.9782e-04 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 9.6224e-04 - mse: 9.6224e-04 - val_loss: 0.1179 - val_mse: 0.1179
Epoch 66/150
33/33 [==============================] - 0s 938us/step - loss: 7.1542e-04 - mse: 7.1542e-04 - val_loss: 0.1222 - val_mse: 0.1222
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 5.5591e-04 - mse: 5.5591e-04 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 68/150
33/33 [==============================] - 0s 970us/step - loss: 6.4647e-04 - mse: 6.4647e-04 - val_loss: 0.1219 - val_mse: 0.1219
Epoch 69/150
33/33 [==============================] - 0s 970us/step - loss: 6.1813e-04 - mse: 6.1813e-04 - val_loss: 0.1182 - val_mse: 0.1182
Epoch 70/150
33/33 [=======

33/33 [==============================] - 0s 1ms/step - loss: 6.2521e-05 - mse: 6.2521e-05 - val_loss: 0.1178 - val_mse: 0.1178
Epoch 123/150
33/33 [==============================] - 0s 2ms/step - loss: 6.1990e-05 - mse: 6.1990e-05 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 6.9641e-05 - mse: 6.9641e-05 - val_loss: 0.1178 - val_mse: 0.1178
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 7.2564e-05 - mse: 7.2564e-05 - val_loss: 0.1185 - val_mse: 0.1185
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 8.2626e-05 - mse: 8.2626e-05 - val_loss: 0.1182 - val_mse: 0.1182
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 8.2276e-05 - mse: 8.2276e-05 - val_loss: 0.1177 - val_mse: 0.1177
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 1.0159e-04 - mse: 1.0159e-04 - val_loss: 0.1184 - val_mse: 0.1184
Epoch 129/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [48]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0026 - mse: 0.0026


[0.002644301624968648, 0.002644301624968648]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [49]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 778us/step - loss: 0.1197 - mse: 0.1197


[0.11967945098876953, 0.11967945098876953]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [50]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 1ms/step - loss: 0.1793 - mse: 0.1793


[0.1792779415845871, 0.1792779415845871]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [51]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

33273.22863986708

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.